In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import re

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-gpu")
    # options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1280, 800)
    driver.set_page_load_timeout(60)
    return driver

def accept_cookies(driver, timeout=5):
    try:
        wait = WebDriverWait(driver, timeout)
        btn = wait.until(EC.element_to_be_clickable((By.ID, "popin_tc_privacy_button_3")))
        btn.click()
        time.sleep(1)
        print("🍪 Cookies acceptés (SG)")
    except Exception:
        print("ℹ️ Aucune pop‑up de cookies détectée")

def extract_agences_from_html_sg(html):
    soup = BeautifulSoup(html, "html.parser")
    agences = []
    for li in soup.select("li.agency"):
        name_div = li.select_one("div.agencymarqueregion")
        addr_div  = li.select_one("div.agencyaddress")
        if not name_div or not addr_div:
            continue

        nom = name_div.get_text(strip=True)
        lignes = [l.strip().strip('"') for l in addr_div.stripped_strings]
        adresse = ", ".join(lignes)
        cp_match = re.search(r"\b(\d{5})\b", adresse)
        code_postal = cp_match.group(1) if cp_match else ""

        agences.append({
            "nom": nom,
            "adresse": adresse,
            "code_postal": code_postal
        })
    return agences

def scrape_department(dept_slug, output_dir="agences_sg"):
    url = f"https://agences.sg.fr/banque-assurance/particulier/{dept_slug}"
    driver = setup_driver()
    print(f"\n🌍 Scraping : {url}")
    driver.get(url)

    accept_cookies(driver)
    # laissez le JS s'exécuter
    time.sleep(3)

    html = driver.page_source
    driver.quit()

    agences = extract_agences_from_html_sg(html)
    df = pd.DataFrame(agences)
    df.drop_duplicates(subset=["nom", "adresse"], inplace=True)

    os.makedirs(output_dir, exist_ok=True)
    path = os.path.join(output_dir, f"{dept_slug}.csv")
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"✅ {len(df)} agences sauvegardées dans {path}")
departments = [
    "agences-ain-01",
    "agences-aisne-02",
    "agences-allier-03",
    "agences-alpes-de-haute-provence-04",
    "agences-hautes-alpes-05",
    "agences-alpes-maritimes-06",
    "agences-ardeche-07",
    "agences-ardennes-08",
    "agences-ariege-09",
    "agences-aube-10",
    "agences-aude-11",
    "agences-aveyron-12",
    "agences-bouches-du-rhone-13",
    "agences-calvados-14",
    "agences-cantal-15",
    "agences-charente-16",
    "agences-charente-maritime-17",
    "agences-cher-18",
    "agences-correze-19",
    "agences-cote-dor-21",
    "agences-cotes-darmor-22",
    "agences-creuse-23",
    "agences-dordogne-24",
    "agences-doubs-25",
    "agences-drome-26",
    "agences-eure-27",
    "agences-eure-et-loir-28",
    "agences-finistere-29",
    "agences-corse-du-sud-2a",
    "agences-haute-corse-2b",
    "agences-gard-30",
    "agences-haute-garonne-31",
    "agences-gers-32",
    "agences-gironde-33",
    "agences-herault-34",
    "agences-ille-et-vilaine-35",
    "agences-indre-36",
    "agences-indre-et-loire-37",
    "agences-isere-38",
    "agences-jura-39",
    "agences-landes-40",
    "agences-loir-et-cher-41",
    "agences-loire-42",
    "agences-haute-loire-43",
    "agences-loire-atlantique-44",
    "agences-loiret-45",
    "agences-lot-46",
    "agences-lot-et-garonne-47",
    "agences-lozere-48",
    "agences-maine-et-loire-49",
    "agences-manche-50",
    "agences-marne-51",
    "agences-haute-marne-52",
    "agences-mayenne-53",
    "agences-meurthe-et-moselle-54",
    "agences-meuse-55",
    "agences-morbihan-56",
    "agences-moselle-57",
    "agences-nievre-58",
    "agences-nord-59",
    "agences-oise-60",
    "agences-orne-61",
    "agences-pas-de-calais-62",
    "agences-puy-de-dome-63",
    "agences-pyrenees-atlantiques-64",
    "agences-hautes-pyrenees-65",
    "agences-pyrenees-orientales-66",
    "agences-bas-rhin-67",
    "agences-haut-rhin-68",
    "agences-rhone-69",
    "agences-haute-saone-70",
    "agences-saone-et-loire-71",
    "agences-sarthe-72",
    "agences-savoie-73",
    "agences-haute-savoie-74",
    "agences-paris-75",
    "agences-seine-maritime-76",
    "agences-seine-et-marne-77",
    "agences-yvelines-78",
    "agences-deux-sevres-79",
    "agences-somme-80",
    "agences-tarn-81",
    "agences-tarn-et-garonne-82",
    "agences-var-83",
    "agences-vaucluse-84",
    "agences-vendee-85",
    "agences-vienne-86",
    "agences-haute-vienne-87",
    "agences-vosges-88",
    "agences-yonne-89",
    "agences-territoire-de-belfort-90",
    "agences-essonne-91",
    "agences-hauts-de-seine-92",
    "agences-seine-saint-denis-93",
    "agences-val-de-marne-94",
    "agences-val-d-oise-95",
]

if __name__ == "__main__":
    # Exemple pour l’Ain
    for slug in departments:
     scrape_department(slug)



🌍 Scraping : https://agences.sg.fr/banque-assurance/particulier/agences-ain-01
🍪 Cookies acceptés (SG)
✅ 13 agences sauvegardées dans agences_sg\agences-ain-01.csv

🌍 Scraping : https://agences.sg.fr/banque-assurance/particulier/agences-aisne-02
🍪 Cookies acceptés (SG)
✅ 13 agences sauvegardées dans agences_sg\agences-aisne-02.csv

🌍 Scraping : https://agences.sg.fr/banque-assurance/particulier/agences-allier-03
🍪 Cookies acceptés (SG)
✅ 6 agences sauvegardées dans agences_sg\agences-allier-03.csv

🌍 Scraping : https://agences.sg.fr/banque-assurance/particulier/agences-alpes-de-haute-provence-04
🍪 Cookies acceptés (SG)
✅ 7 agences sauvegardées dans agences_sg\agences-alpes-de-haute-provence-04.csv

🌍 Scraping : https://agences.sg.fr/banque-assurance/particulier/agences-hautes-alpes-05
🍪 Cookies acceptés (SG)
✅ 1 agences sauvegardées dans agences_sg\agences-hautes-alpes-05.csv

🌍 Scraping : https://agences.sg.fr/banque-assurance/particulier/agences-alpes-maritimes-06
🍪 Cookies acceptés